In [1]:
%load_ext autoreload
%autoreload 2

# Load benchmarks from a csv, look at some useful stats



We examine the impact of the choice of the interpolation scheme

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.market.curves import *

from financepy.products.rates.ibor_benchmarks_report import dataframe_to_benchmarks

We start by defining this useful function

In [4]:
%matplotlib widget
# %matplotlib auto
def plotCurve(curve, start_date, tmax, instr_mat_dates_or_tenor = None, title = ''):
    # years = np.linspace(1/365, tmax, 500)
    years = np.linspace(1/365, tmax, int(365*tmax))
    dates = start_date.add_years(years)
    zero_rates = curve.zero_rate(dates)
    on_fwd_rates = curve.fwd(dates)

    ys_monthly = np.arange(1,tmax*12+1)/12
    monthly_dates = start_date.add_years(ys_monthly)

    if instr_mat_dates_or_tenor is not None:
        # Will plot term forward rates
        # instr_mat_dates_or_tenor could be a list of curve-building
        # isntrument maturities in which case term fwd rates go from the last instrument maturiy date that is less
        # than the plotting date to the plotting date. Or it could be a tenor so that fwd rates cover that tenor 
        # i.e. the term fwd rate for d covers [max(d-tenor,start_date), d]
        # tenor should be a positive tenor

        if isinstance(instr_mat_dates_or_tenor, str):
            neg_tenor = '-' + instr_mat_dates_or_tenor
            start_fwd_dates = [d.add_tenor(neg_tenor) for d in dates]
            start_fwd_dates = [d if d > start_date else start_date for d in start_fwd_dates]
            fwd_rate_label = f'term fwd rates for [d-{instr_mat_dates_or_tenor},d]'
        else:
            instr_mat_dates_or_tenor = [start_date] + instr_mat_dates_or_tenor
            start_fwd_dates = [ max([md  for md in instr_mat_dates_or_tenor if md < d]) for d in dates]
            fwd_rate_label = 'term fwd rates from prev instr mtrty'

        term_fwd_rates = curve.fwd_rate(start_fwd_dates, dates)

    plt.figure(figsize=(8,6))
    plt.plot(years*12, zero_rates*100, '-', label="zero rates")
    plt.plot(years*12, on_fwd_rates*100, '-', label = "ON fwd rates")
    plt.plot(ys_monthly*12, curve.zero_rate(monthly_dates)*100, 'o', label="zero rates (input)")

    if instr_mat_dates_or_tenor is not None:
        plt.plot(years*12, term_fwd_rates*100, '-', label = fwd_rate_label)

    plt.xlabel("Times in months")
    plt.ylabel("Rates (%) - See Legend")
    plt.title(title)
    plt.legend()

### Load csv

In [ ]:
filename = './data/bms_GBP_SONIA_20220901.csv'
dfbm = pd.read_csv(filename, index_col = 0)
dfbm['base_date']=pd.to_datetime(dfbm['base_date'], errors = 'ignore', format = '%d/%m/%Y')
dfbm['start_date']=pd.to_datetime(dfbm['start_date'], errors = 'ignore', format = '%d/%m/%Y')
dfbm['maturity_date']=pd.to_datetime(dfbm['maturity_date'], errors = 'ignore', format = '%d/%m/%Y')
dfbm.head()


### convert to benchmarks

In [ ]:

valuation_date = date.from_datetime(dfbm.loc[0,'base_date'])
cal = CalendarTypes.UNITED_KINGDOM
bms = dataframe_to_benchmarks(dfbm, asof_date=valuation_date,calendar_type=cal)
depos = bms['IborDeposit']
fras = bms['IborFRA']
swaps = bms['IborSwap']

fras.sort( key = lambda fra:fra.maturity_dt)


In [ ]:
remove_mpc_fras = True
if remove_mpc_fras:
    fras_to_use = [f for f in fras if date.datediff(f.start_dt,f.maturity_dt) > 60]
    fras = fras_to_use
len(fras)

In [ ]:
libor_curve = IborSingleCurve(valuation_date, depos, fras, swaps, InterpTypes.LINEAR_ZERO_RATES)
plotCurve(libor_curve, valuation_date, 15, instr_mat_dates_or_tenor='1Y')

### Let's try all available interpolators

In [10]:
# class InterpTypes(Enum):
#     FLAT_FWD_RATES = 1
#     LINEAR_FWD_RATES = 2
#     LINEAR_ZERO_RATES = 4
#     FINCUBIC_ZERO_RATES = 7
#     NATCUBIC_LOG_DISCOUNT = 8
#     NATCUBIC_ZERO_RATES = 9
#     PCHIP_ZERO_RATES = 10
#     PCHIP_LOG_DISCOUNT = 11

# from class PchipInterpolator(CubicHermiteSpline):
#     ``x`` and ``y`` are arrays of values used to approximate some function f,
#     with ``y = f(x)``. The interpolant uses monotonic cubic splines
#     to find the value of new points. (PCHIP stands for Piecewise Cubic
#     Hermite Interpolating Polynomial).

In [ ]:
for interp_type in InterpTypes:
    libor_curve = IborSingleCurve(valuation_date, depos, fras, swaps, interp_type)
    plotCurve(libor_curve, valuation_date, 15, instr_mat_dates_or_tenor='1y', title = interp_type)